keywords: NLP (TALN), classification.

# The Spam Detector

## Description

Madame Esposito développe pour son entreprise un chatbot dans le but de répondre automatiquement à ses nombreux clients. Cependant son programme recoit un grand nombre de message malveillant ou à caractere publicitaire ce qui degrade les performance de son bot en plus d'occasionner des traitements informatiques se répercutant sur sa facture d'electricité.

Madame Esposito vous a contacté afin de créer un programme capable de detecter automatiquement les SPAM. Pour cela, elle a construit un jeux de donnée comportant un ensemble de SMS de type SPAM et NON SPAM (HAM), disponible à l'addresse suivante : http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

Par ailleurs, afin d'intégrer les résultats dans son équipe, elle nous demande les choses suivantes:

Afin d'estimer le cout du devellopement et suivre le projet, une checklist des taches à réaliser doit être rédigé.
https://trello.com/b/JNiVTMvb/spam-detector

Vous devez créer des fonctions pour les diférentes partie de votre code afin de pouvoir les réutiliser facilement
Vous devez effectuer une validation croisé (cross-validation) sur 10 jeux de'apprentissage et de test différent. Le seed doit être fixé à 42 et le jeux de test doit représenter 20% des données.
Comparer au moins trois algorithme de classification en terme de f1 score. Lequel est le plus puissant ?

## Bonus¶

Pouvez améliorer les résultats ?

* est-ce que la lemmatisation améliore les résultats ?
* est-ce que la racinisation (stemming) améliore les résultats ?
Une fois ces étapesréalisées, reproduire la même expérience avec le jeux suivant, représant cette fois des commentaires Youtube : https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection

Les performances sont-elles similaire à ceux obtenus avec le jeux de données précedent ?

Madame esposito souhaite controler si les modéles appris avec le premier jeux de données sont capable de prédire les données de test du deuxième jeu et vice-versa. Réaliser un tableau comparant

* les résultats de prédiction du modeles appris sur les SPAN SMS pour prédire les SPAM commentaire youtube et
* les résultats de prédiction du modeles appris surles SPAM commentaire youtube pour prédire les SPAM SMS youtube

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
# import plotly.express as px # on est pas dans le bon environnement

## nltk
import nltk
# nltk.download() ## à executer qu'une fois
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#stopwords = nltk.corpus.stopwords.words('english')

## sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

### models ML
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn import svm      # Support Vector Machine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier 

### evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


# *************************************************************************
#### Import librairies
import numpy as np
import pandas as pd
### Sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score , precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
#vis
import matplotlib.pyplot as plt 
import seaborn as sns
#os
import time
start_time = time.time()

## Pseudo Randomization of Workgroups

In [48]:
app = 'Anthony Constant Joshua Fatima Julien Bassem Caroline Dan Ines Nidhal Sacia Xavier Roger Hachem Jean-Pierre Myriam Ludo Olivier Pierre-Etienne Wiem Cecilia'.split()
s = app
 
np.random.seed(1)
rapp = [s[i] for i in np.random.choice(21, 21, replace=False)]
for i in range(4):
    e = rapp
    print(e[4*i:4*i+4] + ([e[-1]] if i == 3 else []))

['Myriam', 'Sacia', 'Fatima', 'Pierre-Etienne']
['Olivier', 'Jean-Pierre', 'Ludo', 'Julien']
['Joshua', 'Caroline', 'Dan', 'Constant']
['Hachem', 'Anthony', 'Cecilia', 'Wiem', 'Bassem']


##Proposed Plan

1) *Veille en Traitement du langage + checklist + 45min*

https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32

https://towardsdatascience.com/your-guide-to-natural-language-processing-nlp-48ea2511f6e1

https://towardsdatascience.com/introduction-to-natural-language-processing-for-text-df845750fb63

* Parsing and tokenization ?
PARSING involves undertaking grammatical analysis for the provided sentence.

STEMMING refers to the process of slicing the end or the beginning of words with the intention of removing affixes & suffixes.

LEMMATIZATION has the objective of reducing a word to its base form and grouping together different forms of the same word.

TOKENIZATION is the process of segmenting running text into sentences and words. In essence, it’s the task of cutting a text into pieces called tokens, and at the same time throwing away certain characters, such as punctuation.

* Advantage of TFIDF ?
One approach in NLP is to rescale the frequency of words by how often they appear in all texts (not just the one we are analyzing) so that the scores for frequent words like “the”, that are also frequent across other texts, get penalized. This approach to scoring is called “Term Frequency — Inverse Document Frequency” (TFIDF), and improves the bag of words by weights. Through TFIDF frequent terms in the text are “rewarded” (like the word “they” in our example), but they also get “punished” if those terms are frequent in other texts we include in the algorithm too. On the contrary, this method highlights and “rewards” unique or rare terms considering all texts. Nevertheless, this approach still has no context nor semantics.

* Stops words ?
STOP WORDS removal includes getting rid of common language articles, pronouns and prepositions such as “and”, “the” or “to” in english.

* Vectorization ? Bag of words ?
The BAG OF WORDS is a commonly used model in NLP that allows you to count all words in a piece of text.

In order to address this, scikit-learn provides utilities for the most common ways to extract numerical features from text content, namely:

* tokenizing strings and giving an integer id for each possible token (for instance by using white-spaces and punctuation as token separators)
* counting the occurrences of tokens in each document
* normalizing and weighting with diminishing importance tokens that occur in the majority of samples / documents

Each individual token occurrence frequency (normalized or not) is treated as a feature. The vector of all the token frequencies for a given document is considered as a multivariate sample. A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

In [49]:
import nltk # installation of NLTK
from nltk.corpus import stopwords
nltk.download('stopwords', 'punkt') #comment this line after kernel restart
print(stopwords.words('french')) # French for testing my below quote

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to punkt...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

In [51]:
example_sent = """Et si quelquefois, sur les marches d'un palais, sur l'herbe verte d'un fossé, vous vous réveillez, l'ivresse déjà diminuée ou disparue, demandez au vent, à la vague, à l'étoile, à l'oiseau, à l'horloge; à tout ce qui fuit, à tout ce qui gémit, à tout ce qui roule, à tout ce qui chante, à tout ce qui parle, demandez quelle heure il est. Et le vent, la vague, l'étoile, l'oiseau, l'horloge, vous répondront, il est l'heure de s'enivrer ; pour ne pas être les esclaves martyrisés du temps, enivrez-vous, enivrez-vous sans cesse de vin, de poésie, de vertu, à votre guise."""
stop_words = set(stopwords.words('french'))
word_tokens = word_tokenize(example_sent)
print (word_tokens)

['Et', 'si', 'quelquefois', ',', 'sur', 'les', 'marches', "d'un", 'palais', ',', 'sur', "l'herbe", 'verte', "d'un", 'fossé', ',', 'vous', 'vous', 'réveillez', ',', "l'ivresse", 'déjà', 'diminuée', 'ou', 'disparue', ',', 'demandez', 'au', 'vent', ',', 'à', 'la', 'vague', ',', 'à', "l'étoile", ',', 'à', "l'oiseau", ',', 'à', "l'horloge", ';', 'à', 'tout', 'ce', 'qui', 'fuit', ',', 'à', 'tout', 'ce', 'qui', 'gémit', ',', 'à', 'tout', 'ce', 'qui', 'roule', ',', 'à', 'tout', 'ce', 'qui', 'chante', ',', 'à', 'tout', 'ce', 'qui', 'parle', ',', 'demandez', 'quelle', 'heure', 'il', 'est', '.', 'Et', 'le', 'vent', ',', 'la', 'vague', ',', "l'étoile", ',', "l'oiseau", ',', "l'horloge", ',', 'vous', 'répondront', ',', 'il', 'est', "l'heure", 'de', "s'enivrer", ';', 'pour', 'ne', 'pas', 'être', 'les', 'esclaves', 'martyrisés', 'du', 'temps', ',', 'enivrez-vous', ',', 'enivrez-vous', 'sans', 'cesse', 'de', 'vin', ',', 'de', 'poésie', ',', 'de', 'vertu', ',', 'à', 'votre', 'guise', '.']


In [52]:
filtered_sentence = [w for w in word_tokens if not w in stop_words]  
filtered_sentence = []

In [53]:
for w in word_tokens:  
    if w not in stop_words:  
        filtered_sentence.append(w)  
        
print(filtered_sentence)

['Et', 'si', 'quelquefois', ',', 'marches', "d'un", 'palais', ',', "l'herbe", 'verte', "d'un", 'fossé', ',', 'réveillez', ',', "l'ivresse", 'déjà', 'diminuée', 'disparue', ',', 'demandez', 'vent', ',', 'vague', ',', "l'étoile", ',', "l'oiseau", ',', "l'horloge", ';', 'tout', 'fuit', ',', 'tout', 'gémit', ',', 'tout', 'roule', ',', 'tout', 'chante', ',', 'tout', 'parle', ',', 'demandez', 'quelle', 'heure', '.', 'Et', 'vent', ',', 'vague', ',', "l'étoile", ',', "l'oiseau", ',', "l'horloge", ',', 'répondront', ',', "l'heure", "s'enivrer", ';', 'être', 'esclaves', 'martyrisés', 'temps', ',', 'enivrez-vous', ',', 'enivrez-vous', 'sans', 'cesse', 'vin', ',', 'poésie', ',', 'vertu', ',', 'guise', '.']



### **1. Veille NLT**

https://docs.google.com/document/d/1p1eJt4l0cJZjILpwl9RcKvmSCZAUo17h7RSzA_1-Aa4/edit?ts=5fc6087c#

## Début de l'exercice

### **2) Load, clean and prepare the data**

Vectoriser et netoyer vos données. https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

**Data set dowloaded in local sub folder ./data**

http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Fichier /home/dan/Code/repositoryGIT/simplon-brief7-nltk/SMSSpamCollection.txt

In [54]:
stop_words = set(stopwords.words('english'))  
file1 = open("/home/dan/Code/repositoryGIT/simplon-brief7-nltk/SMSSpamCollection.txt")
line = file1.read() 
words = line.split()
for r in words:  
    if not r in stop_words:  
        appendFile = open('SMSSpam_filtered.txt','a')  
        appendFile.write(" "+r)  
        appendFile.close()

# import w pandas

In [55]:
import pandas as pd
data_df = pd.read_csv("SMSSpamCollection.txt", sep="\t", names = ['label', 'raw_text'])
data_df

,label,raw_text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [56]:
data_df.info

<bound method DataFrame.info of      label                                           raw_text
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [57]:
data_df.describe

<bound method NDFrame.describe of      label                                           raw_text
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [58]:
targets = data_df.label # ham or spam
corpus = data_df.raw_text # the docs
print("Targets :\n", targets.describe(), "\n")
print("Corpus :\n", corpus.describe())

Targets :
 count     5572
unique       2
top        ham
freq      4825
Name: label, dtype: object 

Corpus :
 count                       5572
unique                      5169
top       Sorry, I'll call later
freq                          30
Name: raw_text, dtype: object


### CountVectorizer

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
stopwords = nltk.corpus.stopwords.words("english")
cv = CountVectorizer(stop_words=stopwords)
cv_fit = cv.fit_transform(corpus)

In [60]:
print(cv_fit)

  (0, 3531)	1
  (0, 4311)	1
  (0, 5871)	1
  (0, 2312)	1
  (0, 1301)	1
  (0, 1745)	1
  (0, 3615)	1
  (0, 8417)	1
  (0, 4437)	1
  (0, 1743)	1
  (0, 2036)	1
  (0, 3575)	1
  (0, 1074)	1
  (0, 8207)	1
  (1, 5461)	1
  (1, 4473)	1
  (1, 4279)	1
  (1, 8322)	1
  (1, 5488)	1
  (2, 3335)	1
  (2, 2927)	2
  (2, 8376)	1
  (2, 2153)	1
  (2, 8334)	1
  (2, 3064)	2
  :	:
  (5567, 165)	1
  (5567, 5379)	1
  (5568, 3871)	1
  (5568, 3544)	1
  (5568, 3322)	1
  (5568, 2959)	1
  (5569, 5093)	1
  (5569, 5802)	1
  (5569, 7326)	1
  (5570, 3335)	1
  (5570, 8253)	1
  (5570, 4578)	1
  (5570, 5296)	1
  (5570, 8007)	1
  (5570, 7005)	1
  (5570, 2869)	1
  (5570, 3449)	1
  (5570, 1779)	1
  (5570, 3669)	1
  (5570, 4131)	1
  (5570, 912)	1
  (5570, 1540)	1
  (5571, 5205)	1
  (5571, 7828)	1
  (5571, 6460)	1


### Quel est le type Python de vos données après vectorisation ?

In [61]:
cv_fit

<5572x8577 sparse matrix of type '<class 'numpy.int64'>'
	with 47370 stored elements in Compressed Sparse Row format>

### Quels sont les 10 mots les plus fréquent dans le jeux de données ?

In [62]:
print(len(cv.get_feature_names()), "mots" )

8577 mots


In [63]:
# Transform the sparse matrix into a numpy matrix documents words (mdw)
mdw_cv_fit = cv_fit.toarray()

# Type of the matrix documents terms mdt
print(type(mdw_cv_fit))

# Size of the matrix
print(mdw_cv_fit.shape)

<class 'numpy.ndarray'>
(5572, 8577)


In [64]:
# Frequency of the terms
freq_words = np.sum(mdw_cv_fit,axis=0)
print(freq_words)
# Arg sort
index = np.argsort(freq_words)
print(index)
# Print the matrix words and their frequency
mwf = pd.DataFrame({'word':np.asarray(cv.get_feature_names())[index],'freq':freq_words[index]})
mwf

[10 29  1 ...  1  1  1]
[4288 4821 4818 ... 3483 7992 1804]


,word,freq
0,jot,1
1,mallika,1
2,malarky,1
3,makiing,1
4,maintaining,1
...,...,...
8572,lt,316
8573,gt,318
8574,get,391
8575,ur,391


In [25]:
print('Les dix mots les plus fréquents\n')
print(mwf[-10:])

Les dix mots les plus fréquents

      word  freq
8567  good   247
8568  know   262
8569    go   283
8570  free   284
8571    ok   293
8572    lt   316
8573    gt   318
8574   get   391
8575    ur   391
8576  call   593


## Les moins fréquents ?

In [26]:
print('Les dix mots les moins fréquents\n')
print(mwf[:10])

Les dix mots les moins fréquents

           word  freq
0           jot     1
1       mallika     1
2       malarky     1
3       makiing     1
4   maintaining     1
5        mailed     1
6       mahfuuz     1
7      mahaveer     1
8  magicalsongs     1
9    manageable     1


### Tracer la distribution de la fréquence des mots présents dans le jeux de données. Qu'observez-vous ?

### Aides/propositions

* Stop words avec NLTK

**3) Apprentissage**

Que représente la mesure f1 ?

*In statistical analysis of binary classification, the F-score or F-measure is a measure of a test's accuracy. It is calculated from the precision and recall of the test, where the precision is the number of correctly identified positive results (TP) divided by the number of all positive results (TP + FP), including those not identified correctly, and the recall is the number of correctly identified positive results (TP) divided by the number of all samples that should have been identified as positive (TP + FN).

The F1 score is the harmonic mean of the precision and recall. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is F1 = 2 (precision recall) / (precision + recall)*

**Quelles sont ces avantages sur d'autre mesure tel que la précision le rappel ou l'accuracy ?**

**4) Veille cross validation**  
https://scikit-learn.org/stable/modules/cross_validation.html

Utilisez la méthode de ShuffleSplit pour construire vos jeux de données permettant la validation croisée.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit

In [27]:
from sklearn.model_selection import ShuffleSplit

X = mdw_cv_fit
y = np.array(targets)
rs = ShuffleSplit(test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Number of splits :", rs.get_n_splits(X))
for train_index, test_index in rs.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    rtrain = train_index.shape[0]/ X.shape[0]
    rtest = test_index.shape[0] / X.shape[0]
    print("train =", len(train_index), "documents", "=> train size ratio :", "{:.0%}".format(rtrain))
    print("test =", len(test_index), "documents", "=> test size ratio :", "{:.0%}".format(rtest))

Number of splits : 10
TRAIN: [1978 3989 3935 ... 5226 5390  860] TEST: [3245  944 1044 ... 5556 4205 4293]
train = 4457 documents => train size ratio : 80%
test = 1115 documents => test size ratio : 20%
TRAIN: [1809 4249  448 ... 5215 3867 2077] TEST: [1086  483  855 ... 3609 3902 1851]
train = 4457 documents => train size ratio : 80%
test = 1115 documents => test size ratio : 20%
TRAIN: [1688 2618 2319 ...   53 1245 3166] TEST: [ 577 2279 4517 ... 2743 4177 4220]
train = 4457 documents => train size ratio : 80%
test = 1115 documents => test size ratio : 20%
TRAIN: [  45 2349 4972 ...   98 3693 5304] TEST: [1806 3752 1197 ... 1429 3212 5036]
train = 4457 documents => train size ratio : 80%
test = 1115 documents => test size ratio : 20%
TRAIN: [4670 3060 3614 ... 2703 5187 2965] TEST: [5528 4971  101 ... 3840 4116 1361]
train = 4457 documents => train size ratio : 80%
test = 1115 documents => test size ratio : 20%
TRAIN: [3405 4049 1920 ... 3744 4035 4388] TEST: [5162  590 5271 ... 4270

In [28]:
rs = ShuffleSplit(n_splits=5, train_size=0.5, test_size=.25, random_state=42)
for train_index, test_index in rs.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    rtrain = X_train.shape[0]/ X.shape[0]
    rtest = X_test.shape[0] / X.shape[0]
    print("train =", len(train_index), "documents", "=> train size ratio :", "{:.0%}".format(rtrain))
    print("test =", len(test_index), "documents", "=> test size ratio :", "{:.0%}".format(rtest))

TRAIN: [4281  585 4545 ... 1435 3902 1247] TEST: [3245  944 1044 ... 5536 1657 3875]
train = 2786 documents => train size ratio : 80%
test = 1393 documents => test size ratio : 20%
TRAIN: [ 130  697 3894 ... 2198 3155 4020] TEST: [1086  483  855 ... 3753 5378 4454]
train = 2786 documents => train size ratio : 80%
test = 1393 documents => test size ratio : 20%
TRAIN: [5333 2732 5328 ... 3759 2612 2957] TEST: [ 577 2279 4517 ... 3669  927 3441]
train = 2786 documents => train size ratio : 80%
test = 1393 documents => test size ratio : 20%
TRAIN: [1862 2587 2267 ... 5023 4154  835] TEST: [1806 3752 1197 ... 3505 3792 4080]
train = 2786 documents => train size ratio : 80%
test = 1393 documents => test size ratio : 20%
TRAIN: [2379 3572  417 ... 1105 2521 1581] TEST: [5528 4971  101 ... 2512  148 2695]
train = 2786 documents => train size ratio : 80%
test = 1393 documents => test size ratio : 20%


**5) Train**

Fit the models and compare the performance in a table that show

* the mean of the f1 score
* the standard deviation of the f1 score

Qu'observez vous ?

In [29]:
print("X_train shape :", X_train.shape, "\n", "y_train shape :", y_train.shape)
#print(type(X_train), type(y_train))
print("X_test shape: ", X_test.shape, "\n", "y_test shape :", y_test.shape)
#print(type(X_test), type(y_test))

X_train shape : (4457, 8577) 
 y_train shape : (4457,)
X_test shape:  (1115, 8577) 
 y_test shape : (1115,)


### LogisticRegression

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

LR = LogisticRegression()
clf = LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[966   0]
 [ 16 133]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.89      0.94       149

    accuracy                           0.99      1115
   macro avg       0.99      0.95      0.97      1115
weighted avg       0.99      0.99      0.99      1115



### SVM

In [31]:
from sklearn import svm

X = mdw_cv_fit
y = np.array(targets)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[966   0]
 [ 18 131]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.88      0.94       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Logistic Regression is better than Support Vector Machine

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores

array([0.96155179, 0.96155179, 0.95915224, 0.95671201, 0.96580076])

In [38]:
print("f1_macro with 5 Cross Validation : %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

f1_macro with 5 Cross Validation : 0.9610 (+/- 0.0060)


In [39]:
from sklearn.model_selection import ShuffleSplit
n_samples = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
scores = cross_val_score(clf, X, y, cv=cv, scoring='f1_macro')
scores

array([0.96900147, 0.96043815, 0.96711122, 0.9671907 , 0.96355552])

In [40]:
print("f1_macro with shuffle split : %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

f1_macro with shuffle split : 0.9655 (+/- 0.0061)


### Cross validation, with or without "shuffle shift", doesn't improve the score

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(stop_words=stop_words)
X_TFIDF = vectorizer.fit_transform(corpus)
y_TFIDF = np.array(targets)
X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_TFIDF, test_size=0.2, random_state=42)
LR = LogisticRegression()
clf = LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[965   1]
 [ 45 104]]
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       0.99      0.70      0.82       149

    accuracy                           0.96      1115
   macro avg       0.97      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115



### Logistic regression score is worse with TFIDF normalizaztion¶

###### 

In [ ]:
v_tf = TfidfVectorizer(stop_words=stopwords)
x_tf = v_tf.fit_transform(corpus)

idf_values = dict(zip(v_tf.get_feature_names(), v_tf.idf_))
words_name = v_tf.get_feature_names()

# printing the tfidf vectors
print("shape :",x_tf.shape)
print(x_tf)
# printing the vocabulary
print(v_tf.vocabulary_)

In [2]:
#print(X2.toarray())

print("The Array is : ")
for i in X2:
    print(i, end = ' ')

In [3]:
#stopwords = nltk.corpus.stopwords.words("english")

# TF-IDF Vectorizer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Train pack with tfidf 
x = corpus #corpus
y = targets #target

x_tf_train, x_tf_test, y_train, y_test = train_test_split(x_cv, y, test_size=0.2, random_state=42)                                                   
print('-- Training pack TF-IDF:',"\n")
print('-- x_tf_train rows :',np.size(x_tf_train))
print('-- x_tf_test rows :',np.size(x_tf_test))
print('-- ratio in % :', round(np.size(x_tf_test)/(np.size(x_tf_train)+np.size(x_tf_test))*100,2),"\n") 

#Train pack w/o tfid
x_train, x_test, y_train, y_test = train_test_split(x_tf, y, test_size=0.20, random_state=0)
print('-- Training pack w/o TF-IDF:',"\n")
print('-- x_train rows :',np.size(x_train))
print('-- x_test rows :',np.size(x_test))
print('-- ratio in % :', round(np.size(x_test)/(np.size(x_train)+np.size(x_test))*100,2))

stopwords = nltk.corpus.stopwords.words("english")
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)
X = tfidf_vectorizer.fit_transform(corpus)

#print(vectorizer.get_feature_names())
print('')
print("X shape :", X.shape)
print("X_train shape :", X_train.shape)

NameError: name 'corpus' is not defined

LinearLogistique, svc, random forest

# Logistic Regression

In [ ]:
#Logistic regression base set

loR = LogisticRegression()
loR.fit(x_train, y_train) #Model w/o tf-idf
pred = loR.predict(x_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Prediction base_set:',pred[:100],'\n')
print('-- Recall_score base_set:',round(recall_score(y_test, pred, average ='micro'),3))
print('-- Precision_score base_set:',round(precision_score(y_test, pred, average ='micro'),3),'\n')

In [ ]:
# logistic regression SVM base set
clf = svm.SVC()
clf.fit(x_train,y_train)
predSVM = clf.predict(x_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Prediction base_set:',pred[:100],'\n')
print('-- Recall_score base_set:',round(recall_score(y_test, predSVM, average ='micro'),3))
print('-- Precision_score base_set:',round(precision_score(y_test, predSVM, average ='micro'),3),'\n')

In [ ]:
#Logistic regression base set
loR = LogisticRegression()
loR.fit(x_train, y_train) #Model w/o tf-idf
pred = loR.predict(x_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Prediction base_set:',pred[:100],'\n')
print('-- Recall_score base_set:',round(recall_score(y_test, pred, average ='micro'),3))
print('-- Precision_score base_set:',round(precision_score(y_test, pred, average ='micro'),3),'\n')

In [ ]:
# Logistic regression TF-IDF set
loR_tf = LogisticRegression()
loR_tf.fit(x_tf_train, y_train) #Model w tf-idf
pred_tf = loR_tf.predict(x_tf_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Predition tf-idf set :',pred_tf[:100],'\n')
print('-- Recall_score tf-idf set:',round(recall_score(y_test, pred_tf),3))
print('-- Precision_score tf-idf set:',round(precision_score(y_test, pred_tf),3))

In [ ]:
# logistic regression SVM base set
clf = svm.SVC()
clf.fit(x_train,y_train)
predSVM = clf.predict(x_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Prediction base_set:',pred[:100],'\n')
print('-- Recall_score base_set:',round(recall_score(y_test, predSVM, average ='micro'),3))
print('-- Precision_score base_set:',round(precision_score(y_test, predSVM, average ='micro'),3),'\n')

In [ ]:
# logistic regression SVM TF-IDF set
clf = svm.SVC()
clf.fit(x_tf_train,y_train)
predSVM = clf.predict(x_tf_test)
print("-- Loading time : %s secondes. --" % (time.time() - start_time))
print('-- Prediction TF-IDF_set:',pred[:100],'\n')
print('-- Recall_score TF-IDF_set:',round(recall_score(y_test, predSVM, average ='micro'),3))
print('-- Precision_score TF-IDF_set:',round(precision_score(y_test, predSVM, average ='micro'),3),'\n')

# Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm